In [2]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from zoneinfo import ZoneInfo
import re

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("googleplaystore.csv")

# -------------------------
# DATA CLEANING
# -------------------------

# Clean Installs
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Clean Reviews & Rating
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# Convert Size to MB
def size_to_mb(x):
    try:
        if isinstance(x, str):
            if "M" in x:
                return float(x.replace("M", ""))
            elif "k" in x:
                return float(x.replace("k", "")) / 1024
        return None
    except:
        return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# Convert Last Updated to datetime
df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")

# -------------------------
# APPLY FILTERS
# -------------------------

filtered = df[
    (df["Rating"] >= 4.2) &
    (~df["App"].str.contains(r"\d", na=False)) &
    (df["Category"].str.startswith(("T", "P"), na=False)) &
    (df["Reviews"] > 1000) &
    (df["Size_MB"].between(20, 80))
]

# -------------------------
# GROUP MONTHLY INSTALLS
# -------------------------

filtered["YearMonth"] = filtered["Last Updated"].dt.to_period("M")
monthly = (
    filtered.groupby(["YearMonth", "Category"])["Installs"]
    .sum()
    .reset_index()
)

monthly["YearMonth"] = monthly["YearMonth"].astype(str)

# Pivot for stacked area
pivot_df = monthly.pivot(index="YearMonth", columns="Category", values="Installs").fillna(0)

# Cumulative installs
cumulative_df = pivot_df.cumsum()

# -------------------------
# LEGEND TRANSLATION
# -------------------------

translations = {
    "Travel & Local": "Voyage et Local",
    "Productivity": "Productividad",
    "Photography": "写真"
}

# -------------------------
# TIME CONDITION (4PM–6PM IST)
# -------------------------

current_time = datetime.now(ZoneInfo("Asia/Kolkata"))
current_hour = current_time.hour

print("Current IST Time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

if 16 <= current_hour < 18 and not cumulative_df.empty:

    fig = go.Figure()

    for category in cumulative_df.columns:

        display_name = translations.get(category, category)

        # Calculate MoM growth
        growth = pivot_df[category].pct_change()

        # Increase color intensity if any month >25% growth
        high_growth = (growth > 0.25).any()

        opacity = 0.9 if high_growth else 0.6

        fig.add_trace(go.Scatter(
            x=cumulative_df.index,
            y=cumulative_df[category],
            mode='lines',
            stackgroup='one',
            name=display_name,
            opacity=opacity
        ))

    fig.update_layout(
        title="Cumulative Installs by Category (Filtered)\nVisible Only Between 4 PM IST and 6 PM IST",
        xaxis_title="Month",
        yaxis_title="Cumulative Installs",
        template="plotly_white"
    )

    fig.show()

else:
    print("⚠ Dashboard Locked")
    print("Stacked Area Chart is visible ONLY between 4 PM IST and 6 PM IST.")


C:\Users\GOD\AppData\Local\Temp\ipykernel_15768\3730474787.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["YearMonth"] = filtered["Last Updated"].dt.to_period("M")


Current IST Time: 2026-02-16 09:19:53
⚠ Dashboard Locked
Stacked Area Chart is visible ONLY between 4 PM IST and 6 PM IST.
